In [ ]:
# Here to split raw data into two sets, training set & testing set (Holdout 1/3)

In [41]:
import numpy as np
import pandas as pd

In [42]:
# Read in csv file of raw traffic record
rec_df_raw = pd.read_csv("./data/tmdb_5000_features.csv")

In [45]:
# Delete unwanted features, features: language cast_num cast_0 crew_num director genre_0 campany_0 country
rec_df =  rec_df_raw[['language', 'cast_num', 'cast_0', 'crew_num', 'director', 'genre_0', 'company_0', 'country']]

In [47]:
# Raw 4790 movies, training 3194, testing 1596
train_df = rec_df.head(3194)
test_df = rec_df.head(1596)

In [48]:
train_df.to_csv('chp5_classifiers/train_set.csv', index=False)
test_df.to_csv('chp5_classifiers/test_set.csv', index=False)